# Reddit API

In [1]:
%load_ext dotenv
%dotenv

In [2]:
import praw # reddit API wrapper
import re
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from datetime import datetime

In [3]:
reddit = praw.Reddit(client_id=os.environ.get('REDDIT_CLIENT_ID'),
                     client_secret=os.environ.get('REDDIT_CLIENT_SECRET'),
                     user_agent=os.environ.get('REDDIT_USER_AGENT'))

In [149]:
def users_from_post(post):
    '''
    Given a post, return all users that participate (through commenting or posting)
    Input: post
    Output: set(users)
    '''
    users = set()
    users.add(post.author.name)
    post.comments.replace_more(limit=None)
    for comment in post.comments.list():
        if comment.author:
            users.add(comment.author.name)
    return users

In [134]:
def recent_users(sub, limit=None):
    '''
    Given a subreddit, return all users who recently participated (through commenting or posting)
    Input: subreddit name
    Output: set(users)
    '''
    users = set()
    for post in reddit.subreddit(sub).new(limit=None):
        try:
            test = users_from_post(post)
            users = users.union(test)
        except Exception as e:
            print(e, post.url)
    return users

In [187]:
def recent_subreddits(user):
    '''
    Given a reddit username, return subreddits they have been active in recently.
    Input: Reddit username
    Output: list of dicts [{user, subreddit, timestamp}]
    '''
    sub_activity = []
    for comment in reddit.redditor(user).comments.new(limit=None):
        sub = comment.subreddit.display_name
        time = comment.created_utc
        sub_activity.append({'user':user,
                             'subreddit':sub,
                             'created_at':datetime.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')})
    return sub_activity

In [152]:
users_co = recent_users('competitiveoverwatch')

In [136]:
len(users)

4162

In [166]:
len(users_co)

7923

In [188]:
data = recent_subreddits('RangerFM')

In [190]:
pd.DataFrame(data)

,created_at,subreddit,user
0,2019-11-07 20:52:55,Competitiveoverwatch,RangerFM
1,2019-11-05 13:51:12,guitars,RangerFM
2,2019-11-04 17:24:52,Bluegrass,RangerFM
3,2019-10-30 00:42:12,asheville,RangerFM
4,2019-10-24 12:45:18,Competitiveoverwatch,RangerFM
5,2019-10-23 19:45:58,Competitiveoverwatch,RangerFM
6,2019-10-23 19:07:14,Competitiveoverwatch,RangerFM
7,2019-10-23 17:39:06,Competitiveoverwatch,RangerFM
8,2019-10-23 16:53:17,RepTime,RangerFM
9,2019-10-21 17:12:59,Bluegrass,RangerFM
